In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
def softmax(x):
    exp = np.exp(x)
    return exp / np.sum(exp)

In [3]:
def rescorla_wagner(q_val, epsilon_app, epsilon_wth, action, reward):
    if action == 1:
        return q_val + epsilon_app * (reward - q_val)

    return q_val + epsilon_wth * (reward - q_val)

In [4]:
def reward(r_t, rho_rew, rho_pun):
    if r_t > 0: return rho_rew
    if r_t < 0: return -rho_pun
    return 0

In [5]:
def bias(a, bias_app, bias_wth):
    return bias_app if a == 1 else bias_wth

In [6]:
def log_likelihood(cues, actions, rewards, epsilon_app, epsilon_wth, rho_rew, rho_pun, bias_app, bias_wth):
    n_stimuli = len(set(cues))
    n_actions = len(set(actions))

    q_vals = np.zeros((n_stimuli, n_actions))

    log_likelihood = 0

    for t, a_t in enumerate(actions):
        s_t = cues[t] - 1
        r_t = reward(rewards[t], rho_rew, rho_pun)

        qs = q_vals[s_t] + [ bias_wth, bias_app ]

        probs = softmax(qs)
        log_likelihood += np.log(probs[a_t])

        # Update the Q-values using Rescorla-Wagner, add action bias
        q_vals[s_t, a_t] = rescorla_wagner(
            q_val = q_vals[s_t, a_t],
            epsilon_app = epsilon_app,
            epsilon_wth = epsilon_wth,
            action = a_t,
            reward = r_t
        )

    return log_likelihood

In [7]:
data = pd.read_csv("gen_data.csv")

In [8]:
def loss(params, cues, actions, rewards):
    epsilon_app, epsilon_wth, rho_rew, rho_pun, bias_app, bias_wth = params
    return -log_likelihood(cues, actions, rewards, epsilon_app, epsilon_wth, rho_rew, rho_pun, bias_app, bias_wth)

In [9]:
epsilon_bounds = (0.0000001, 0.99999)
beta_bounds = (0.0001, 9.9999)
bias_bounds = (-0.99999, 0.99999)

In [10]:
min_loss = []

for subject_id in data.ID.unique():
    subject = data[data.ID == subject_id]

    cues = subject.cue.tolist()
    actions = subject.pressed.tolist()
    rewards = subject.outcome.tolist()

    res = minimize(
        fun = loss,
        x0 = [0.5, 0.5, 5, 5, 0, 0],
        bounds = [epsilon_bounds, epsilon_bounds, beta_bounds, beta_bounds, bias_bounds, bias_bounds],
        args = (cues, actions, rewards),
        method = "Nelder-Mead"
    )
    
    min_loss.append(res.fun)

In [11]:
np.sum(min_loss) # 2682

2682.4008837954207